In [2]:
import numpy as np
import scipy
import sympy as sp

# 1) Правило Армихо

$x^0=(-1,1)$

$f:R^2\rightarrow R, f(x)=2x_1^2+x_1x_2+3x^2_2$

In [3]:
def func(x):
    return 2 * x[0] ** 2 + x[0] * x[1] + 3 * x[1] ** 2

def f_grad(x):
    grad = np.zeros_like(x)
    grad[0] = 4 * x[0] + x[1]
    grad[1] = x[0] + 6 * x[1] 
    return grad

def grad_descent(x, alpha, eps, theta, precision): 
    count = 0
    flag=True 
    while flag:
        last_x = x
        x_alpha = x - alpha * f_grad(x)
        # условие Армихо
        while func(x_alpha) > func(x) + eps * np.cross(f_grad(x), x_alpha ):
            alpha = theta * alpha
            x_alpha = x - alpha * f_grad(x)
            
        x = x - alpha * f_grad(x)
        count += 1
        if np.linalg.norm(x - last_x) < precision:
            flag=False
                
    print("Iters num: " + str(count)) 
    return x


x0 = np.array([-1.0, 1.0])
alpha = 1 
eps = 0.5 
theta = 0.5
result = grad_descent(x0, alpha=alpha, eps=eps, theta=theta, precision=0.001)
print(f"Solution x1={result[0]} x2={result[1]}")

Iters num: 15
Solution x1=-0.00010639708489179611 x2=-0.0002568652853369713


# 2) Метод Ньютона

$x^0=(1,1)$

$f:R^2\rightarrow R, f(x)=x_1^2+e^{x^2_2}$

In [5]:
def func(x):
    return np.array(x[0] ** 2 + np.exp(x[1] ** 2))

def jacob(x):
    grad = np.zeros_like(x)
    grad[0] = 2 * x[0]
    grad[1] = 2 * x[1] *np.exp(x[1]**2) 
    return grad

def hessian(x):
    grad = np.zeros_like(x)
    grad[0] = 2
    grad[1] = 2 * np.exp(x[1] ** 2) * (1 + 2 * x[1] ** 2)
    return grad


def newton(x, precision): 
    count = 0
    flag=True 
    while flag:
        count += 1 
        x_last = x
        x=x-jacob(x)/hessian(x)
        if np.linalg.norm(x - x_last) < precision: 
            flag=False

    print("Iters num: " + str(count)) 
    return x

x0 = np.array([1.0, 1.0])
result = newton(x0, precision=1e-12)
print(f"Solution x1={result[0]} x2={result[1]}") 
print(scipy.optimize.newton(func, x0, maxiter=1000))

Iters num: 7
Solution x1=0.0 x2=0.0
[0.04924812 0.04924812]


# 3) Метод сопряженных градиентов

# 4) Метод условного градиента

# 5) Метод квадратичного штрафа

$f(x)=2x^2_1+(x_2-1)^2\rightarrow min, x \in D=\{x \in R^2 | 2x_1+x_2=2\} $

In [6]:
def func(x):
    return 2 * x[0] ** 2 + (x[1] - 1) ** 2

def F(x):
    return 2 * x[0] + x[1]

def penalty(x):
    return 0.5 * np.linalg.norm(F(x)) ** 2

def quadratic(x,N=10000): 
    c_k = 1
    count = 0 
    iters=0 
    flag=True 
    eps=1e-5 
    while flag:
        curr_func = lambda x: func(x) + c_k * penalty(x) 
        x_old=x.copy()
        data = scipy.optimize.minimize(curr_func, x)
        iters+=data.nit
        x=data.x 
        c_k += 1e5
        count += 1
        if np.linalg.norm(x-x_old) < eps or count>N:
            flag=False
    print("Inner Iters num: " + str(iters))
    return x
x0 = np.array([-1.0, 1.0])
result = quadratic(x0)
print(f"Solution x1={result[0]:2f} x2={result[1]:2f}")

Inner Iters num: 11
Solution x1=-0.333341 x2=0.666686


# 6) Симплекс метод 

In [9]:
c = [-1, 3, 5, 1]
A = [[1,4,4, 1], [1, 7, 8, 2]] 
b = [5, 9]

def func(x):
    return sum(c[:-1]*x)

x1_bounds = (0, None) 
x2_bounds = (0, None)
x3_bounds = (0, None) 
x4_bounds = (0, None)
res = scipy.optimize.linprog(c, A_eq=A, b_eq=b, bounds=[x1_bounds, x2_bounds, x3_bounds,x4_bounds ], method='simplex')

print(f'x1={res[0]}, x2={res[1]}, x3={res[2]}')
print(f'Minimum: {func(res[:-1])}')

/var/folders/zs/yzy0xdy166x3hv5mvqzqrnlm0000gn/T/ipykernel_8011/1998074309.py:12: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res = scipy.optimize.linprog(c, A_eq=A, b_eq=b, bounds=[x1_bounds, x2_bounds, x3_bounds,x4_bounds ], method='simplex')


KeyError: 0

In [10]:
res

     con: array([0., 0.])
     fun: 3.0
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([1., 0., 0., 4.])

In [12]:
m= sp.Matrix([[1, 4, 4, 1,5], [1, 7, 8, 2,9],
              [-1, 3, 5, 1,0]]) 
basis=np.array([1,0,1,0])
basis_indeces=np.where(basis==1)
ones_view_matrix, pivots = m.rref(normalize_last=True) 
ones_view_matrix=np.array(ones_view_matrix).astype(np.float32) 
print(ones_view_matrix)
res=ones_view_matrix[:,-1]
print(f'x1={res[0]}, x2={res[1]}, x3={res[2]},')
print(f'Minimum: {func(res)}')

[[  1.   0.   0.   1.  17.]
 [  0.   1.   0.  -1. -16.]
 [  0.   0.   1.   1.  13.]]
x1=17.0, x2=-16.0, x3=13.0,
Minimum: 0.0
